In [1]:
%cd ".."
import pyLDAvis
import json
import pandas as pd
import doc_embedding.moyenne as moyenne
from gensim import models
from sklearn.cluster import k_means

d:\users\atero\Github\projet-statistique-2022


C:\Users\atero\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\sparse\sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


Chargement des documents

In [2]:
with open('data/docs.json', encoding = "utf8") as f:
    docs: list[str] = json.load(f)
print(f"Nombre de documents : {len(docs)}")

Nombre de documents : 9501


In [3]:
with open('data/liste_lemmes.txt') as f:
    vocabulaire = f.readlines()

# Suppression des retours chariots sur chaque ligne
for i in range(len(vocabulaire)):
    vocabulaire[i] = vocabulaire[i].replace('\n', '')

print(f"Taille du vocabulaire : {len(vocabulaire)}")

Taille du vocabulaire : 7231


Calcul du nombre d'apparition de chaque mot, on conserve seulement ceux présents un certain nombre de fois

In [4]:
# Nombre de fois où chaque mot apparaît
term_frequency_corpus: dict = {mot: 0 for mot in vocabulaire}
for doc in docs:
    for mot in doc:
        term_frequency_corpus[mot] += 1

term_freq_min: int = 15
vocabulaireMin: list = [mot for mot in vocabulaire if term_frequency_corpus[mot] >= term_freq_min]
term_frequency_corpus_min = {mot: term_frequency_corpus[mot] for mot in vocabulaireMin}

Chargement des valeurs génériques à toutes les représentations de modèles

In [5]:
args_pyLDA = {'doc_lengths': [len(doc) for doc in docs], 'vocab': vocabulaireMin, 'term_frequency': list(term_frequency_corpus_min.values()), 'n_jobs': -1}

# Modèle CBOW

Chargement de modèle et de labels clusters

In [6]:
w2vec: models.Word2Vec = models.Word2Vec.load("./data/w2vec.bin")
ev = moyenne.word_emb_vers_doc_emb_moyenne(docs, w2vec.wv)

In [7]:
res = k_means(ev, n_clusters = 4)

In [8]:
clustersDocs: list[int] = res[1]
nomsClusters = list(set(clustersDocs))
K: int = len(nomsClusters)

Calcul des probabilités empiriques d'appartenance des mots aux divers clusters

In [9]:
# Probabilité que chaque mot appartienne à un cluster / topic
term_frequency_topics: pd.DataFrame = pd.DataFrame(columns = vocabulaireMin, index = nomsClusters, dtype = float)
term_frequency_topics.loc[:,:] = 0

for i, doc in enumerate(docs):
    clusterDoc: int = clustersDocs[i]
    for mot in doc:
        if mot in vocabulaireMin:
            term_frequency_topics.loc[clusterDoc, mot] += 1 / term_frequency_corpus[mot]


Probabilité d'appartenance d'un document à un cluster. Dans notre cadre l'appartenance est certaine (probabilité de 0 ou de 1)

In [10]:
# Probabilité qu'un document appartienne à un cluster / topic (soit 0 soit 1 pour notre cas)
topic_docs_matrice: pd.DataFrame = pd.DataFrame(columns = nomsClusters, index = range(len(docs)), dtype = float)
topic_docs_matrice.loc[:,:] = 0

for i in range(len(docs)):
    topic_docs_matrice.loc[i,clustersDocs[i]] = 1

In [11]:
visualisation = pyLDAvis.prepare(topic_term_dists = term_frequency_topics.to_numpy(), doc_topic_dists = topic_docs_matrice.to_numpy(), **args_pyLDA)
pyLDAvis.save_html(visualisation, "data/LDAvis/CBOW.html")

C:\Users\atero\AppData\Local\Programs\Python\Python310\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [12]:
pyLDAvis.display(visualisation)

# Modèle Skip-Gram

Chargement de modèle et de labels clusters

In [13]:
w2vec: models.Word2Vec = models.Word2Vec.load("./data/w2vec.bin")
ev = moyenne.word_emb_vers_doc_emb_moyenne(docs, w2vec.wv)

In [14]:
res = k_means(ev, n_clusters = 4)

In [15]:
clustersDocs: list[int] = res[1]
nomsClusters = list(set(clustersDocs))
K: int = len(nomsClusters)

Calcul des probabilités empiriques d'appartenance des mots aux divers clusters

In [16]:
# Probabilité que chaque mot appartienne à un cluster / topic
term_frequency_topics: pd.DataFrame = pd.DataFrame(columns = vocabulaireMin, index = nomsClusters, dtype = float)
term_frequency_topics.loc[:,:] = 0

for i, doc in enumerate(docs):
    clusterDoc: int = clustersDocs[i]
    for mot in doc:
        if mot in vocabulaireMin:
            term_frequency_topics.loc[clusterDoc, mot] += 1 / term_frequency_corpus[mot]


Probabilité d'appartenance d'un document à un cluster. Dans notre cadre l'appartenance est certaine (probabilité de 0 ou de 1)

In [17]:
# Probabilité qu'un document appartienne à un cluster / topic (soit 0 soit 1 pour notre cas)
topic_docs_matrice: pd.DataFrame = pd.DataFrame(columns = nomsClusters, index = range(len(docs)), dtype = float)
topic_docs_matrice.loc[:,:] = 0

for i in range(len(docs)):
    topic_docs_matrice.loc[i,clustersDocs[i]] = 1

In [18]:
visualisation = pyLDAvis.prepare(topic_term_dists = term_frequency_topics.to_numpy(), doc_topic_dists = topic_docs_matrice.to_numpy(), **args_pyLDA)
pyLDAvis.save_html(visualisation, "data/LDAvis/Skip_Gram.html")

C:\Users\atero\AppData\Local\Programs\Python\Python310\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [19]:
pyLDAvis.display(visualisation)

# Modèle GloVe

Chargement de modèle et de labels clusters

In [20]:
w2vec: models.Word2Vec = models.Word2Vec.load("./data/w2vec.bin")
ev = moyenne.word_emb_vers_doc_emb_moyenne(docs, w2vec.wv)

In [21]:
res = k_means(ev, n_clusters = 4)

In [22]:
clustersDocs: list[int] = res[1]
nomsClusters = list(set(clustersDocs))
K: int = len(nomsClusters)

Calcul des probabilités empiriques d'appartenance des mots aux divers clusters

In [23]:
# Probabilité que chaque mot appartienne à un cluster / topic
term_frequency_topics: pd.DataFrame = pd.DataFrame(columns = vocabulaireMin, index = nomsClusters, dtype = float)
term_frequency_topics.loc[:,:] = 0

for i, doc in enumerate(docs):
    clusterDoc: int = clustersDocs[i]
    for mot in doc:
        if mot in vocabulaireMin:
            term_frequency_topics.loc[clusterDoc, mot] += 1 / term_frequency_corpus[mot]

Probabilité d'appartenance d'un document à un cluster. Dans notre cadre l'appartenance est certaine (probabilité de 0 ou de 1)

In [24]:
# Probabilité qu'un document appartienne à un cluster / topic (soit 0 soit 1 pour notre cas)
topic_docs_matrice: pd.DataFrame = pd.DataFrame(columns = nomsClusters, index = range(len(docs)), dtype = float)
topic_docs_matrice.loc[:,:] = 0

for i in range(len(docs)):
    topic_docs_matrice.loc[i,clustersDocs[i]] = 1

In [25]:
visualisation = pyLDAvis.prepare(topic_term_dists = term_frequency_topics.to_numpy(), doc_topic_dists = topic_docs_matrice.to_numpy(), **args_pyLDA)
pyLDAvis.save_html(visualisation, "data/LDAvis/GloVe.html")

C:\Users\atero\AppData\Local\Programs\Python\Python310\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [26]:
pyLDAvis.display(visualisation)